<a href="https://colab.research.google.com/github/mizscrystal/Information-Gain/blob/main/InformationGain_GininSplit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

Creating the Dataset

In [ ]:
# Defining a simple dataset
attribute_names =  ['positive web review', 'Fuel Economy','Low Noise']
class_name = 'Test Drive'
# creating the Car test dataset
data_Car = {'positive web review':['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes','yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes','no','no','no','no'],
        'Fuel Economy':['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes','yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes','no','no','no','no','no','no','no','no','no'],
        'Low Noise':['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes','yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes','yes','no','no','no','no','no','no','no','no'],
        'Test Drive':['yes', 'no', 'no', 'yes', 'no', 'yes', 'no', 'yes', 'yes','yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'no','no','yes','yes','no','no','no','no','yes','no']}

# converting the dataset into a dataframe
Test_Car_dataset = pd.DataFrame(data = data_Car)

In [ ]:
Test_Car_dataset

,positive web review,Fuel Economy,Low Noise,Test Drive
0,yes,yes,yes,yes
1,yes,yes,yes,no
2,yes,yes,yes,no
3,yes,yes,yes,yes
4,yes,yes,yes,no
5,yes,yes,yes,yes
6,yes,yes,yes,no
7,yes,yes,yes,yes
8,yes,yes,yes,yes
9,yes,yes,yes,yes


Gini Impurity is a measurement used to build Decision Trees to determine how the features of a dataset should split nodes to form the tree. More precisely, the Gini Impurity of a dataset is a number between 0-0.5, which indicates the likelihood of new, random data being misclassified if it were given a random class label according to the class distribution in the dataset.

In [ ]:
# STEP 1: Calculate gini(D)
def gini_impurity (value_counts):
    n = value_counts.sum()
    p_sum = 0
    for key in value_counts.keys():
        p_sum = p_sum  +  (value_counts[key] / n ) * (value_counts[key] / n ) 
    gini = 1 - p_sum
    return gini

class_value_counts = Test_Car_dataset[class_name].value_counts()
print(f'Number of samples in each class is:\n{class_value_counts}')

gini_class = gini_impurity(class_value_counts)
print(f'\nGini Impurity of the class is {gini_class:.3f}')


Number of samples in each class is:
yes    16
no     13
Name: Test Drive, dtype: int64

Gini Impurity of the class is 0.495


In [ ]:
# STEP 2: 
# Calculating  gini impurity for the attiributes
def gini_split_a(attribute_name):
    attribute_values = Test_Car_dataset[attribute_name].value_counts()
    gini_A = 0 
    for key in attribute_values.keys():
        df_k = Test_Car_dataset[class_name][Test_Car_dataset[attribute_name] == key].value_counts()
        n_k = attribute_values[key]
        n = Test_Car_dataset.shape[0]
        gini_A = gini_A + (( n_k / n) * gini_impurity(df_k))
    return gini_A

gini_attiribute ={}
for key in attribute_names:
    gini_attiribute[key] = gini_split_a(key)
    print(f'Gini for {key} is {gini_attiribute[key]:.3f}')


Gini for positive web review is 0.466
Gini for Fuel Economy is 0.452
Gini for Low Noise is 0.471


In [ ]:
# STEP 3: 
# Compute Gini gain values to find the best split
# An attribute has maximum Gini gain is selected for splitting.

min_value = min(gini_attiribute.values())
print('The minimum value of Gini Impurity : {0:.3} '.format(min_value))
print('The maximum value of Gini Gain     : {0:.3} '.format(1-min_value))

selected_attribute = min(gini_attiribute.keys())
print('The selected attiribute is: ', selected_attribute)


The minimum value of Gini Impurity : 0.452 
The maximum value of Gini Gain     : 0.548 
The selected attiribute is:  Fuel Economy


This calculation measured the impurity of the split, and the feature with the lowest impurity would determine the best feature for splitting the current node. From our calculation, the best feature is Fuel Economy

Information gain is the main key that is used by Decision Tree Algorithms to construct a Decision Tree.

In [ ]:
from __future__ import division
from math import log


def entropy(pi):
    '''
    return the Entropy of a probability distribution:
    entropy(p) = − SUM (Pi * log(Pi) )
    defintion:
            entropy is a metric to measure the uncertainty of a probability distribution.
    entropy ranges between 0 to 1
    Low entropy means the distribution varies (peaks and valleys).
    High entropy means the distribution is uniform.
    '''

    total = 0
    for p in pi:
        p = p / sum(pi)
        if p != 0:
            total += p * log(p, 2)
        else:
            total += 0
    total *= -1
    return total


def gain(d, a):
    '''
    return the information gain:
    gain(D, A) = entropy(D)−􏰋 SUM ( |Di| / |D| * entropy(Di) )
    '''

    total = 0
    for v in a:
        total += sum(v) / sum(d) * entropy(v)

    gain = entropy(d) - total
    return gain


TestDrive = [16, 13] # Yes, No


# attribute, number of members (feature)
positivewebreview = [
    [15, 1],  # overcase
    [10, 3]  # sunny
]
FuelEconomy = [
    [13, 3],  # hot
    [7, 6]  # cool

]
LowNoise = [
    [13, 3],  # high
    [8, 5]   # normal
]

print(gain(TestDrive, positivewebreview))
print(gain(TestDrive, FuelEconomy))
print(gain(TestDrive, LowNoise))

0.45681184731710556
0.16178894771582852
0.17725036205054034


What is the best Information Gain (IG)?
The feature with the highest Information Gain is selected as the best one. Hence from our calcluation it is Positive Web Review